In [8]:
import numpy as np
import pandas as pd
import math
import os
dpath = os.path.join('.', 'data')

# Расчет несущей способности буровой сваи

# Исходные данные

**Грунты:**

In [9]:
NL = -0.35 # отметка поверхности природного рельефа
DL = -1.29 # отметка планировки
soil = pd.read_table(os.path.join(dpath, 'Грунт.csv'), sep=';')

**Параметры сваи:**

In [10]:
d = 880 # диаметр сваи, мм
Yc = 1 # Коэффициент условий работы сваи

# Расчёт

In [11]:
def Rf(tbl, il, z):
    tbl.index = pd.to_numeric(tbl.index)
    new_index = np.unique(list(tbl.index) + [z])
    new_tbl = tbl.reindex(new_index).interpolate('index').loc[z]
    new_tbl.index = pd.to_numeric(new_tbl.index)
    new_id = np.unique(list(pd.to_numeric(new_tbl.index)) + [il])
    rf = new_tbl.reindex(new_id).interpolate('index').loc[il]
    return rf
def fi(t, il, z):
    if t == 'песчаный':
        tbl = pd.read_table(os.path.join(dpath, 'Песчаные_7.3.csv'), sep=';', index_col=0)
    else:
        tbl = pd.read_table(os.path.join(dpath, 'Глинистые_7.3.csv'), sep=';', index_col=0)
    f = Rf(tbl, il, z)
    return f

def ri(t, il, z):
    if t == 'Глинистые_7.8':
        tbl = pd.read_table(os.path.join(dpath, t + '.csv'), sep=';', index_col=0).interpolate()
        r = Rf(tbl, il, z)
        print('R=', round(r, 3), 'кПа')
    else:
        tbl_1_2 = pd.read_table(os.path.join(dpath, t + '_1_2' + '.csv'), sep=';').transpose()
        tbl_1_2.index = pd.to_numeric(tbl_1_2.index)
        new_index = np.unique(list(tbl_1_2.index) + [il])
        alpha_1_2 = tbl_1_2.reindex(new_index).interpolate('index').loc[il]
        print('alpha_1=', round(alpha_1_2[0], 3))
        print('alpha_2=', round(alpha_1_2[1], 3))
        tbl_3 = pd.read_table(os.path.join(dpath, t + '_3' + '.csv'), sep=';', index_col=0)
        alpha_3 = Rf(tbl_3, il, z / d * 1000)
        print('alpha_3=', round(alpha_3, 3))
        tbl_4 = pd.read_table(os.path.join(dpath, t + '_4' + '.csv'), sep=';', index_col=0)
        alpha_4 = Rf(tbl_4, il, d / 1000)
        print('alpha_4=', round(alpha_4, 3))
        Y1_ = soil['Y'][n]
        print('Y1_=', round(Y1_, 3), 'кН/м^3')
        Y1 = sum(soil['Y'].head(-1) * soil['h'].head(-1)) / sum(soil['h'].head(-1))
        print('Y1=', round(Y1, 3), 'кН/м^3')
        print('h=', round(z, 3), 'м')
        r = 0.75 * alpha_4 * (alpha_1_2[0] * Y1_ * d / 1000 + alpha_1_2[1] * alpha_3 * Y1 * z)
        print('R=', round(r, 3), 'кПа')
    return r

In [12]:
if DL > NL:
    cut = 'подсыпка'
elif DL < NL:
    cut = 'срезка'
else:
    cut = 'нет'
cutv = DL - NL

if abs(cutv) < 3.0:
    CL = NL
elif cutv < 0:
    CL = DL + 3.0
else:
    CL = DL - 3.0
soil['h'] = np.append(soil['Верх'].head(-1).to_numpy() - soil['Верх'].tail(-1).to_numpy(), 0)
soil['z'] = abs(soil['Верх'].to_numpy() - soil['h'].to_numpy()/2 - CL)

In [13]:
n = len(soil)-1
A = math.pi * (d/1000) ** 2 / 4
u = math.pi * (d/1000)
Lp = soil['Верх'][0] - soil['Верх'][n]
pr = soil['IL/fi'][n]
if soil['Грунт'][n] == 'песчаный':
    if cut == 'срезка':
        zr = DL - soil['Верх'][n]
    else:
        zr = NL - soil['Верх'][n]
    tab = 'Песчаные_7.7'
else:
    zr = soil['z'][0] - soil['h'][0] / 2 + Lp
    tab = 'Глинистые_7.8'
f = np.zeros(n)
R = ri(tab, soil['IL/fi'][n], zr)
for i in range(0, n):
    f[i] = fi(soil['Грунт'][i], soil['IL/fi'][i], soil['z'][i])
soil['f/R'] = np.append(f, R)
skin = u * sum(soil['YR'] * soil['f/R'] * soil['h'] * soil['k'])
base = soil['YR'][n] * R * A * soil['k'][n]
Fd = Yc * (base + skin)

alpha_1= 29.5
alpha_2= 54.75
alpha_3= 0.61
alpha_4= 0.264
Y1_= 16.2 кН/м^3
Y1= 17.35 кН/м^3
h= 30.24 м
R= 3549.402 кПа


# Результаты

In [14]:
print('Отметка поверхности природного рельефа NL=', NL)
print('Планировка:', cut, np.round(cutv, 2), 'м')
print('Условная отметка:', CL, 'м')
print('Коэффициент условий работы сваи Yc=', Yc)
print('Параметры слоёв грунта:')
print(soil)
print('Параметры сваи:')
print('Площадь опирания:', round(A, 3), 'м^2')
print('Периметр сечения ствола:', round(u, 3), 'м')
print('Длина:', Lp, 'м')
print('глубина заложения нижнего конца сваи', round(zr, 2), 'м')
print('Коэффициент надежности:', round(Yc, 3))
print('Результаты:')
print('Боковая поверхность несёт:', round(skin, 2), 'кН')
print('Конец сваи несёт:', round(base, 2), 'кН')
print('Несущая способность сваи Fd=:', round(Fd, 2), 'кН')

Отметка поверхности природного рельефа NL= -0.35
Планировка: срезка -0.94 м
Условная отметка: -0.35 м
Коэффициент условий работы сваи Yc= 1
Параметры слоёв грунта:
   ИГЭ   Верх      Грунт   IL/fi     Y  k   YR     h      z          f/R
0   2' -15.13   песчаный   0.200  16.9  1  0.7  1.72  15.64    72.896000
1   3' -16.85   песчаный   0.200  17.6  1  0.7  1.30  17.15    75.010000
2   4' -18.15  глинистый   0.167  18.0  1  0.7  1.76  18.68    77.152000
3   4' -19.91  глинистый   0.167  18.0  1  0.7  1.76  20.44    79.616000
4   4' -21.67  глинистый   0.167  18.0  1  0.7  1.76  22.20    82.080000
5   4' -23.43  глинистый   0.167  18.0  1  0.7  1.76  23.96    84.544000
6   4' -25.19  глинистый   0.167  18.0  1  0.7  1.76  25.72    87.008000
7   5' -26.95   песчаный   0.200  16.2  1  0.7  2.00  27.60    89.640000
8   5' -28.95   песчаный   0.200  16.2  1  0.7  2.00  29.60    92.440000
9   5' -30.95   песчаный   0.200  16.2  1  0.7  0.58  30.89    94.246000
10  5' -31.53   песчаный  30.000 